# Transformer-PyTorch 基础概念教程

本教程将介绍 Transformer 架构的基础概念，并演示如何使用 Transformer-PyTorch 库。

## 目录
1. [环境设置](#环境设置)
2. [Transformer 架构概述](#transformer-架构概述)
3. [注意力机制](#注意力机制)
4. [位置编码](#位置编码)
5. [编码器和解码器](#编码器和解码器)
6. [完整模型](#完整模型)
7. [实际应用](#实际应用)

## 环境设置

首先导入必要的库并设置环境。

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 导入 Transformer-PyTorch
from transformer_pytorch import (
    TransformerConfig,
    Transformer,
    get_config,
    set_seed,
    get_device,
    print_model_info
)
from transformer_pytorch.core import (
    MultiHeadAttention,
    TransformerEmbedding,
    create_encoder,
    create_decoder
)

# 设置随机种子和设备
set_seed(42)
device = get_device()
print(f"使用设备: {device}")

# 设置绘图样式
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## Transformer 架构概述

Transformer 是一种基于注意力机制的神经网络架构，由编码器和解码器组成。

### 主要组件：
- **多头注意力机制** (Multi-Head Attention)
- **位置编码** (Positional Encoding)
- **前馈网络** (Feed Forward Network)
- **层归一化** (Layer Normalization)
- **残差连接** (Residual Connection)

In [ ]:
# 创建一个小型 Transformer 配置
config = TransformerConfig(
    vocab_size=1000,
    d_model=128,
    num_heads=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    d_ff=256,
    max_seq_len=32,
    dropout=0.1
)

print("Transformer 配置:")
print(f"词汇表大小: {config.vocab_size:,}")
print(f"模型维度: {config.d_model}")
print(f"注意力头数: {config.num_heads}")
print(f"编码器层数: {config.num_encoder_layers}")
print(f"解码器层数: {config.num_decoder_layers}")

# 估算参数量
params = config.estimate_parameters()
print(f"\n估算参数量: {params['total_M']}")

## 注意力机制

注意力机制是 Transformer 的核心，它允许模型在处理序列时关注不同位置的信息。

### 缩放点积注意力公式：
$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

In [ ]:
# 演示多头注意力机制
d_model, num_heads = 128, 4
attention = MultiHeadAttention(d_model, num_heads, dropout=0.0)

# 创建示例输入
batch_size, seq_len = 2, 8
x = torch.randn(batch_size, seq_len, d_model)

print(f"输入形状: {x.shape}")

# 前向传播
output, attention_weights = attention(x, x, x)

print(f"输出形状: {output.shape}")
print(f"注意力权重形状: {attention_weights.shape}")
print(f"参数数量: {sum(p.numel() for p in attention.parameters()):,}")

In [ ]:
# 可视化注意力权重
# 取第一个样本、第一个头的注意力权重
attn_matrix = attention_weights[0, 0].detach().numpy()

plt.figure(figsize=(8, 6))
sns.heatmap(
    attn_matrix,
    annot=True,
    fmt='.3f',
    cmap='Blues',
    xticklabels=[f'K{i}' for i in range(seq_len)],
    yticklabels=[f'Q{i}' for i in range(seq_len)]
)
plt.title('多头注意力权重矩阵 (第1个头)')
plt.xlabel('键 (Key) 位置')
plt.ylabel('查询 (Query) 位置')
plt.tight_layout()
plt.show()

# 验证注意力权重是否归一化
row_sums = attn_matrix.sum(axis=1)
print(f"\n注意力权重行和: {row_sums}")
print(f"是否归一化: {np.allclose(row_sums, 1.0)}")

## 位置编码

由于注意力机制本身不包含位置信息，Transformer 使用位置编码来为序列中的每个位置添加位置信息。

### 正弦余弦位置编码公式：
- $PE_{(pos, 2i)} = \sin(pos / 10000^{2i/d_{model}})$
- $PE_{(pos, 2i+1)} = \cos(pos / 10000^{2i/d_{model}})$

In [ ]:
# 演示位置编码
vocab_size, d_model, max_len = 1000, 128, 32
embedding = TransformerEmbedding(
    vocab_size=vocab_size,
    d_model=d_model,
    max_len=max_len,
    position_encoding_type='sinusoidal',
    dropout=0.0
)

# 创建示例词元序列
token_ids = torch.randint(1, vocab_size, (1, 16))
embedded = embedding(token_ids)

print(f"词元ID: {token_ids[0][:8].tolist()}...")
print(f"嵌入输出形状: {embedded.shape}")

# 提取位置编码
pos_encoding = embedding.position_encoding.encoding.pe[0, :16, :].detach().numpy()

# 可视化位置编码
plt.figure(figsize=(12, 6))

# 显示前64个维度的位置编码
plt.subplot(1, 2, 1)
sns.heatmap(
    pos_encoding[:, :64].T,
    cmap='RdBu_r',
    center=0,
    xticklabels=range(0, 16, 2),
    yticklabels=range(0, 64, 8)
)
plt.title('正弦余弦位置编码')
plt.xlabel('位置')
plt.ylabel('维度')

# 显示几个维度的位置编码曲线
plt.subplot(1, 2, 2)
for dim in [0, 1, 2, 3]:
    plt.plot(pos_encoding[:, dim], label=f'维度 {dim}')
plt.title('位置编码曲线')
plt.xlabel('位置')
plt.ylabel('编码值')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 编码器和解码器

### 编码器
编码器由多个相同的层组成，每层包含：
1. 多头自注意力机制
2. 残差连接和层归一化
3. 前馈网络
4. 残差连接和层归一化

### 解码器
解码器也由多个相同的层组成，每层包含：
1. 掩码多头自注意力机制
2. 残差连接和层归一化
3. 多头交叉注意力机制
4. 残差连接和层归一化
5. 前馈网络
6. 残差连接和层归一化

In [ ]:
# 演示编码器
d_model, num_heads, d_ff, num_layers = 128, 4, 256, 2
encoder = create_encoder(d_model, num_heads, d_ff, num_layers)

# 输入数据
x = torch.randn(2, 10, d_model)
print(f"编码器输入形状: {x.shape}")

# 编码
encoded = encoder(x)
print(f"编码器输出形状: {encoded.shape}")
print(f"编码器参数数量: {sum(p.numel() for p in encoder.parameters()):,}")

In [ ]:
# 演示解码器
decoder = create_decoder(d_model, num_heads, d_ff, num_layers)

# 目标序列和编码器输出
tgt = torch.randn(2, 8, d_model)
memory = encoded  # 使用编码器的输出作为记忆

print(f"解码器输入形状: {tgt.shape}")
print(f"记忆形状: {memory.shape}")

# 解码
decoded = decoder(tgt, memory)
print(f"解码器输出形状: {decoded.shape}")
print(f"解码器参数数量: {sum(p.numel() for p in decoder.parameters()):,}")

## 完整模型

现在让我们创建并使用一个完整的 Transformer 模型。

In [ ]:
# 创建完整的 Transformer 模型
model = Transformer(config).to(device)

# 打印模型信息
print_model_info(model)

# 准备示例数据
batch_size = 2
src_len, tgt_len = 12, 10

src = torch.randint(1, config.vocab_size, (batch_size, src_len)).to(device)
tgt = torch.randint(1, config.vocab_size, (batch_size, tgt_len)).to(device)

print(f"\n输入数据:")
print(f"源序列形状: {src.shape}")
print(f"目标序列形状: {tgt.shape}")
print(f"源序列示例: {src[0][:8].tolist()}...")
print(f"目标序列示例: {tgt[0][:8].tolist()}...")

In [ ]:
# 前向传播
model.eval()
with torch.no_grad():
    output = model(src, tgt)

print(f"模型输出:")
print(f"Logits 形状: {output['logits'].shape}")
print(f"最后隐藏状态形状: {output['last_hidden_state'].shape}")

# 计算损失和困惑度
logits = output['logits']
shift_logits = logits[..., :-1, :].contiguous()
shift_labels = tgt[..., 1:].contiguous()

loss = F.cross_entropy(
    shift_logits.view(-1, shift_logits.size(-1)),
    shift_labels.view(-1),
    ignore_index=config.pad_token_id
)
perplexity = torch.exp(loss)

print(f"\n性能指标:")
print(f"损失: {loss.item():.4f}")
print(f"困惑度: {perplexity.item():.4f}")

## 实际应用

让我们演示一些实际的应用场景。

In [ ]:
# 1. 序列到序列翻译（模拟）
print("=== 序列到序列翻译 ===")

# 模拟源语言和目标语言句子
src_sentence = torch.tensor([[1, 10, 25, 67, 89, 2]]).to(device)  # "BOS hello world how are EOS"
tgt_sentence = torch.tensor([[1, 15, 30, 45, 60]]).to(device)     # "BOS 你好 世界 怎么 样"

print(f"源句子: {src_sentence[0].tolist()}")
print(f"目标句子: {tgt_sentence[0].tolist()}")

with torch.no_grad():
    translation_output = model(src_sentence, tgt_sentence)
    
# 获取每个位置的预测
predictions = torch.argmax(translation_output['logits'], dim=-1)
print(f"模型预测: {predictions[0].tolist()}")

In [ ]:
# 2. 文本生成
print("\n=== 文本生成 ===")

# 使用模型生成文本
src_context = torch.tensor([[1, 20, 35, 50, 2]]).to(device)  # 上下文

print(f"输入上下文: {src_context[0].tolist()}")

with torch.no_grad():
    generated = model.generate(
        src=src_context,
        max_length=15,
        temperature=0.8,
        do_sample=True
    )

print(f"生成序列: {generated[0].tolist()}")
print(f"生成长度: {generated.size(1)}")

In [ ]:
# 3. 注意力分析
print("\n=== 注意力分析 ===")

# 获取详细的注意力权重
with torch.no_grad():
    detailed_output = model(src_sentence, tgt_sentence, return_dict=True)

# 分析编码器注意力
if detailed_output['encoder_attentions'] is not None:
    encoder_attn = detailed_output['encoder_attentions'][0]  # 第一层
    print(f"编码器注意力形状: {encoder_attn.shape}")
    
    # 可视化第一个头的注意力
    attn_head_0 = encoder_attn[0, 0].cpu().numpy()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        attn_head_0,
        annot=True,
        fmt='.2f',
        cmap='Blues',
        xticklabels=[f'Pos{i}' for i in range(attn_head_0.shape[1])],
        yticklabels=[f'Pos{i}' for i in range(attn_head_0.shape[0])]
    )
    plt.title('编码器自注意力权重 (第1层, 第1头)')
    plt.xlabel('键位置')
    plt.ylabel('查询位置')
    plt.tight_layout()
    plt.show()

# 分析解码器交叉注意力
if detailed_output['decoder_cross_attentions'] is not None:
    cross_attn = detailed_output['decoder_cross_attentions'][0]  # 第一层
    print(f"\n解码器交叉注意力形状: {cross_attn.shape}")
    
    # 可视化第一个头的交叉注意力
    cross_attn_head_0 = cross_attn[0, 0].cpu().numpy()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cross_attn_head_0,
        annot=True,
        fmt='.2f',
        cmap='Reds',
        xticklabels=[f'Src{i}' for i in range(cross_attn_head_0.shape[1])],
        yticklabels=[f'Tgt{i}' for i in range(cross_attn_head_0.shape[0])]
    )
    plt.title('解码器交叉注意力权重 (第1层, 第1头)')
    plt.xlabel('源序列位置')
    plt.ylabel('目标序列位置')
    plt.tight_layout()
    plt.show()

## 总结

在本教程中，我们学习了：

1. **Transformer 架构的基本组件**：注意力机制、位置编码、编码器、解码器
2. **如何使用 Transformer-PyTorch 库**：创建模型、配置参数、前向传播
3. **注意力机制的工作原理**：缩放点积注意力、多头注意力
4. **位置编码的重要性**：正弦余弦编码为序列添加位置信息
5. **实际应用场景**：序列到序列翻译、文本生成、注意力分析

### 下一步
- 探索更复杂的模型配置
- 学习如何训练 Transformer 模型
- 了解不同的注意力机制变体
- 实现具体的 NLP 任务

### 参考资源
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762) - 原始 Transformer 论文
- [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/) - 可视化教程
- [Transformer-PyTorch 文档](https://transformer-pytorch.readthedocs.io/) - 详细 API 文档